# Combined TMB and MSI Analysis

This notebook provides a framework for combined analysis of Tumor Mutational Burden (TMB) and Microsatellite Instability (MSI).

## Background

**Microsatellite Instability (MSI)** and **Tumor Mutational Burden (TMB)** are both important biomarkers in cancer:

- **MSI**: Measures instability in repetitive DNA sequences (microsatellites)
  - Detected by MSIsensor-pro (this workflow)
  - MSI-High tumors often respond well to immunotherapy
  
- **TMB**: Measures the total number of mutations per megabase in the tumor genome
  - Typically calculated from variant calling results (VCF files)
  - High TMB is associated with better immunotherapy response
  
MSI-High tumors often have high TMB, but not always. This notebook helps analyze both biomarkers together.

## Note on TMB Calculation

This workflow currently focuses on MSI detection. To calculate TMB, you would need:
1. Variant calling results (VCF files) from a variant caller like Mutect2, VarScan, or Strelka
2. The callable genome size (typically exome size ~30-50 Mb or WGS ~3000 Mb)
3. Filtering for somatic mutations (not germline variants)

**Future enhancement**: Add variant calling and TMB calculation to this workflow.


In [ ]:
# Standard preamble - imports and configuration
from project_utils.notebookpreamble import *

## Setup and Load MSI Results

In [ ]:
# Setup paths
paths = setup_paths()
config = load_config()

# Load MSI results
workflow_mode = 'tumor_only' if 'baseline' in config.get('aliases', {}) else 'tumor_normal'
species = config['ref']['species']
build = config['ref']['build']
release = config['ref']['release']
genome_version = f"genome.dna.{species}.{build}.{release}"

results_file = paths['results'] / f"{workflow_mode}.{genome_version}.all_samples.tsv"
msi_results = load_msi_results(results_file)
msi_results['msi_status'] = classify_msi_status(msi_results['msi_score'], threshold_high=0.2)

print(f"Loaded {len(msi_results)} samples")
msi_results.head()

## TMB Calculation Placeholder

This section demonstrates how to calculate TMB if you have variant calling results.

### Expected TMB Input Format

TMB is typically calculated from VCF files:
```python
# Example TMB calculation (requires VCF files):
# tmb = total_somatic_mutations / callable_genome_size_mb
```

### Simulated TMB Data for Demonstration

For this demonstration, we'll simulate TMB values based on MSI status (real data would come from variant calling).

In [ ]:
# DEMONSTRATION ONLY: Simulate TMB values
# In practice, TMB would be calculated from actual variant calls
np.random.seed(42)

# MSI-High tumors tend to have higher TMB
def simulate_tmb(msi_status):
    """Simulate TMB values based on MSI status for demonstration."""
    if msi_status == 'MSI-High':
        return np.random.lognormal(mean=3.5, sigma=0.5)  # Higher TMB
    elif msi_status == 'MSI-Low':
        return np.random.lognormal(mean=2.0, sigma=0.6)  # Medium TMB
    else:  # MSS
        return np.random.lognormal(mean=1.5, sigma=0.7)  # Lower TMB

# Add simulated TMB column
msi_results['tmb_simulated'] = msi_results['msi_status'].apply(simulate_tmb)

print("\nNote: TMB values are SIMULATED for demonstration purposes")
print("In a real analysis, TMB would be calculated from variant calling results\n")
msi_results[['group', 'msi_status', 'msi_score', 'tmb_simulated']].head(10)

## TMB Classification

Common TMB thresholds (may vary by context):
- **TMB-High**: ≥ 10 mutations/Mb (some studies use ≥ 20)
- **TMB-Low**: < 10 mutations/Mb

In [ ]:
# Classify TMB status
def classify_tmb(tmb_value, threshold=10):
    """Classify TMB status."""
    return 'TMB-High' if tmb_value >= threshold else 'TMB-Low'

msi_results['tmb_status'] = msi_results['tmb_simulated'].apply(classify_tmb)

print("TMB Status Distribution:")
print(msi_results['tmb_status'].value_counts())

## Combined MSI and TMB Analysis

### Correlation Analysis

In [ ]:
# Calculate correlation between MSI score and TMB
from scipy import stats

correlation, p_value = stats.pearsonr(msi_results['msi_score'], msi_results['tmb_simulated'])
print(f"Pearson correlation between MSI score and TMB: {correlation:.3f}")
print(f"P-value: {p_value:.3e}")
print(f"\nNote: This correlation uses SIMULATED TMB data")

### Combined Classification

In [ ]:
# Create combined classification
def combined_biomarker_status(row):
    """Combine MSI and TMB status."""
    msi = 'MSI-H' if row['msi_status'] == 'MSI-High' else 'MSI-L/MSS'
    tmb = 'TMB-H' if row['tmb_status'] == 'TMB-High' else 'TMB-L'
    return f"{msi}/{tmb}"

msi_results['combined_status'] = msi_results.apply(combined_biomarker_status, axis=1)

print("Combined MSI/TMB Status:")
print(msi_results['combined_status'].value_counts())
print(f"\nCross-tabulation:")
pd.crosstab(msi_results['msi_status'], msi_results['tmb_status'], margins=True)

## Visualizations

### MSI vs TMB Scatter Plot

In [ ]:
# Scatter plot: MSI score vs TMB
fig, ax = plt.subplots(figsize=(10, 8))

# Color by combined status
for status in msi_results['combined_status'].unique():
    subset = msi_results[msi_results['combined_status'] == status]
    ax.scatter(subset['msi_score'], subset['tmb_simulated'], 
              label=status, alpha=0.7, s=100)

# Add threshold lines
ax.axvline(x=0.2, color='red', linestyle='--', alpha=0.5, label='MSI-High threshold')
ax.axhline(y=10, color='blue', linestyle='--', alpha=0.5, label='TMB-High threshold')

ax.set_xlabel('MSI Score', fontsize=12)
ax.set_ylabel('TMB (mutations/Mb) [SIMULATED]', fontsize=12)
ax.set_title('MSI Score vs Tumor Mutational Burden', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Distribution Comparison

In [ ]:
# Box plots comparing TMB across MSI status groups
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# TMB by MSI status
ax1 = axes[0]
msi_results.boxplot(column='tmb_simulated', by='msi_status', ax=ax1)
ax1.set_title('TMB Distribution by MSI Status [SIMULATED]')
ax1.set_xlabel('MSI Status')
ax1.set_ylabel('TMB (mutations/Mb)')
plt.suptitle('')

# MSI score by TMB status
ax2 = axes[1]
msi_results.boxplot(column='msi_score', by='tmb_status', ax=ax2)
ax2.set_title('MSI Score Distribution by TMB Status')
ax2.set_xlabel('TMB Status [SIMULATED]')
ax2.set_ylabel('MSI Score')
plt.suptitle('')

plt.tight_layout()
plt.show()

### Heatmap of Combined Status

In [ ]:
# Create a heatmap showing the relationship between MSI and TMB
fig, ax = plt.subplots(figsize=(8, 6))

# Create contingency table
contingency = pd.crosstab(msi_results['msi_status'], msi_results['tmb_status'])

# Plot heatmap
sns.heatmap(contingency, annot=True, fmt='d', cmap='YlOrRd', ax=ax)
ax.set_title('MSI Status vs TMB Status - Sample Counts [TMB SIMULATED]')
ax.set_ylabel('MSI Status')
ax.set_xlabel('TMB Status')

plt.tight_layout()
plt.show()

## Clinical Interpretation

### Immunotherapy Response Predictions

Both MSI-High and TMB-High are associated with better response to immune checkpoint inhibitors.

In [ ]:
# Categorize samples by predicted immunotherapy response
def predict_immunotherapy_response(row):
    """Predict likely response to immunotherapy based on MSI and TMB."""
    if row['msi_status'] == 'MSI-High' or row['tmb_status'] == 'TMB-High':
        return 'Likely Responder'
    elif row['msi_status'] == 'MSI-Low' and row['tmb_status'] == 'TMB-Low':
        return 'Uncertain'
    else:
        return 'Less Likely to Respond'

msi_results['predicted_response'] = msi_results.apply(predict_immunotherapy_response, axis=1)

print("Predicted Immunotherapy Response:")
print(msi_results['predicted_response'].value_counts())
print(f"\nNote: Predictions based on SIMULATED TMB data")
print("Clinical decisions should be based on actual TMB calculations and other clinical factors")

## Summary Report

In [ ]:
# Generate comprehensive summary
summary_df = msi_results[['group', 'msi_score', 'msi_status', 'tmb_simulated', 
                          'tmb_status', 'combined_status', 'predicted_response']].copy()
summary_df.columns = ['Sample', 'MSI Score', 'MSI Status', 'TMB (simulated)', 
                     'TMB Status', 'Combined Status', 'Predicted Response']

print("="*80)
print("COMBINED MSI AND TMB ANALYSIS SUMMARY")
print("="*80)
print(f"\nTotal samples: {len(summary_df)}")
print(f"\nMSI Status Breakdown:")
for status in ['MSI-High', 'MSI-Low', 'MSS']:
    count = (summary_df['MSI Status'] == status).sum()
    print(f"  {status}: {count} ({count/len(summary_df)*100:.1f}%)")

print(f"\nTMB Status Breakdown (SIMULATED):")
for status in summary_df['TMB Status'].unique():
    count = (summary_df['TMB Status'] == status).sum()
    print(f"  {status}: {count} ({count/len(summary_df)*100:.1f}%)")

print(f"\nPredicted Immunotherapy Response:")
for response in summary_df['Predicted Response'].value_counts().index:
    count = (summary_df['Predicted Response'] == response).sum()
    print(f"  {response}: {count} ({count/len(summary_df)*100:.1f}%)")

print("\n" + "="*80)
print("\nIMPORTANT: TMB values in this analysis are SIMULATED")
print("For real TMB analysis, implement variant calling and TMB calculation in the workflow")
print("="*80)

summary_df

## Export Results

In [ ]:
# Export combined results
output_file = paths['results'] / f"{workflow_mode}.{genome_version}.msi_tmb_combined.tsv"
msi_results.to_csv(output_file, sep='\t', index=False)
print(f"Combined results saved to: {output_file}")
print("\nNote: TMB values are simulated and should be replaced with real calculations")

## Next Steps for Real TMB Integration

To add actual TMB calculation to this workflow:

1. **Add variant calling**: Integrate a somatic variant caller (e.g., Mutect2, VarScan2, Strelka2)
2. **Filter variants**: Filter for high-quality somatic variants
3. **Calculate TMB**: Count variants and divide by callable genome size
4. **Create TMB rule**: Add Snakemake rule to calculate TMB from VCF files
5. **Update this notebook**: Load real TMB values instead of simulated data

Example TMB calculation rule structure:
```python
rule calculate_tmb:
    input:
        vcf="results/variants/{sample}.somatic.filtered.vcf",
        callable_bed="resources/callable_regions.bed"
    output:
        tmb="results/tmb/{sample}.tmb.txt"
    script:
        "scripts/calculate_tmb.py"
```